In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os.path as op
from IPython.display import display

print("*"*5+"Welcome"+"*"*5)
while True:
    path_str=input("Enter path of Airbnb listings database for a particular city:")
    if op.isfile(path_str)==True and ((path_str.split("."))[-1]).lower()=="csv":
        try:
            df_raw=pd.read_csv(path_str)
        except:
            print("Error in reading file....")
            continue
        else:
            break
    else:
        print("Invalid file path....")
        continue
print("The database is read succefully...")
display(df_raw.head(5))
print("Raw database has {0} rows x {1} columns".format(df_raw.shape[0],df_raw.shape[1]))
display(df_raw.head(5))


def Price_convert(p):
    try:
        return float(p.replace("$", ""))
    except:
        pass
           
def max_denisty(feature):
    lst=[((df.describe()).loc["25%",feature])-((df.describe()).loc["min",feature]),((df.describe()).loc["50%",feature])-((df.describe()).loc["25%",feature]),((df.describe()).loc["75%",feature])-((df.describe()).loc["50%",feature]),((df.describe()).loc["max",feature])-((df.describe()).loc["75%",feature])]
    return lst.index(min(lst))


print("Raw database has {0} rows x {1} columns".format(df_raw.shape[0],df_raw.shape[1]))
print("Our analysis will focus on factors such as price, availability, location, and property type,\nSo we will select these features from raw database and create new database")
selected_features=['host_location','host_neighbourhood','neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude','property_type', 'room_type','bedrooms', 'beds','price','availability_365','review_scores_rating']
df=df_raw.loc[:,selected_features]
display(df.head(5))
print("New database has {0} rows x {1} columns".format(df.shape[0],df.shape[1]))
print("Database information.....")
df.info()

print("Cleaning data...")
print("Detect the particalur city")
city=df["host_location"].value_counts().index[0]
print("The city is: {0}".format(city))
print("Drop all rows that don't belong to {0}".format(city))
df=df[df["host_location"]==str(city)]
print("New database has {0} rows x {1} columns".format(df.shape[0],df.shape[1]))
print("Total duplicated rows are {0} row\n".format(df.duplicated().sum(axis=0)))
print("Remove duplicated rows....")
df.drop_duplicates(inplace=True)
print("New database has {0} rows x {1} columns".format(df.shape[0],df.shape[1]))
miss=df.isnull().sum()
print("Total missing cells are {0} cell\n".format(miss.sum(axis=0)))
print("Details missing values count for each feature:\n")
print(miss)
for column in df.columns:
    if (df[column].isnull().sum())>=(0.9*df.shape[0]):
        print("{0} feature has almost all values are null so we will drop this feature".format(column))
        df.drop(labels=column,axis=1,inplace=True)
print("Convert (price) feature to numerical value")
df["price($)"]=df["price"].apply(Price_convert)






numerical_features=df.select_dtypes(np.number).columns
categorical_feature=df.select_dtypes(["object","bool"]).columns
print("Replace nun with median value in numerical features....")
for column in numerical_features:
    df[column].fillna(value=(df[column].median()),inplace=True)
print("Replace nun with mode value in categorical features....")
for column in categorical_feature:
    df[column].fillna(value=(df[column].value_counts().index[0]),inplace=True)
print(df.isnull().sum())    
print("Total duplicated rows are {0} row\n".format(df.duplicated().sum(axis=0)))
print("Remove duplicated rows....")
df.drop_duplicates(inplace=True)
print("New database has {0} rows x {1} columns".format(df.shape[0],df.shape[1]))
print("Show ")
display(df.head(5))

print("Database descreption.....")
display(df.describe())

price_range= [(df.describe()).loc["min","price($)"],(df.describe()).loc["25%","price($)"],(df.describe()).loc["50%","price($)"],(df.describe()).loc["75%","price($)"],(df.describe()).loc["max","price($)"]]
def price_range_func(price):
    if price<price_range[1] and price>=price_range[0]:
        return str(price_range[0])+"$"+" ~ "+str(price_range[1])+"$"
    elif price<price_range[2] and price>=price_range[1]:
        return str(price_range[1])+"$"+" ~ "+str(price_range[2])+"$"
    elif price<price_range[3] and price>=price_range[2]:
        return str(price_range[2])+"$"+" ~"+str(price_range[3])+"$"
    elif price<price_range[4] and price>=price_range[3]:
        return str(price_range[3])+"$" +" ~"+str(price_range[4])+"$"
    

df["price range($)"]=df["price($)"].apply(price_range_func)


availability_365_range= [(df.describe()).loc["min","availability_365"],(df.describe()).loc["25%","availability_365"],(df.describe()).loc["50%","availability_365"],(df.describe()).loc["75%","availability_365"],(df.describe()).loc["max","availability_365"]]
def availability_365_range_func(availability_365):
    if availability_365<availability_365_range[1] and availability_365>=availability_365_range[0]:
        return str(availability_365_range[0])+" ~"+str(availability_365_range[1])
    elif availability_365<availability_365_range[2] and availability_365>=availability_365_range[1]:
        return str(availability_365_range[1])+" ~"+str(availability_365_range[2])
    elif availability_365<availability_365_range[3] and availability_365>=availability_365_range[2]:
        return str(availability_365_range[2])+" ~"+str(availability_365_range[3])
    elif availability_365<availability_365_range[4] and availability_365>=availability_365_range[3]:
        return str(availability_365_range[3])+" ~"+str(availability_365_range[4])
    

df["availability_365_range"]=df["availability_365"].apply(availability_365_range_func)

numerical_features=(pd.DataFrame(df.select_dtypes(np.number))).columns.tolist()
plt.figure(figsize=(10,10))
sns.heatmap(data=(pd.DataFrame(df[numerical_features])).corr(),annot=True)
plt.show()
print("There are a high relation between (bedrooms) and (beds)")
print("There are a relation between price and (bedrooms)(beds) features")

sns.distplot(df["bedrooms"])
plt.show()
print("From the abvoe graph the most rooms have \033[1m {0} bedroom(s) \033[0m and about 75% from rooms have up to \033[1m {1} bedroom(s) \033[0m".format((df["bedrooms"].value_counts().index.tolist())[0],((df.describe()).loc["75%","bedrooms"])))




sns.distplot(df["beds"])
plt.show()
print("From the abvoe graph the most rooms have \033[1m {0} bed(s) \033[0m and about 75% from rooms have up to \033[1m {1} bed(s) \033[0m".format((df["beds"].value_counts().index.tolist())[0],((df.describe()).loc["75%","beds"])))
sns.distplot(df["availability_365"])
plt.show()
print("From the abvoe graph the most rooms have avalilability days from \033[1m {0} to {1} days \033[0m ".format((df.describe()).iloc[(max_denisty("availability_365")+3),4],(df.describe()).iloc[(max_denisty("availability_365")+4),4]))
match max_denisty("availability_365"):
        case 0:
            print("\033[1m So,The city rooms is so busy \033[0m") 
        case 3:
            print("\033[1m So,No problem at all in rooms availability in the city \033[0m")
        case default:
            print("So,The rooms availability is moderate in the city")
sns.distplot(df["price($)"])
plt.show()
print("From the abvoe graph the price of the most rooms from \033[1m {0}$ to {1}$ \033[0m".format((df.describe()).iloc[(max_denisty("price($)")+3),6],(df.describe()).iloc[(max_denisty("price($)")+4),6]))
match max_denisty("price($)"):
        case 0:
            print("\033[1m So,The price of the most rooms is cheap \033[0m") 
        case 3:
            print("\033[1m So,The price of the most rooms is expensive \033[0m")
        case default:
            print("\033[1m So,The price of the most rooms is moderate price \033[0m")
sns.distplot(df["review_scores_rating"])
plt.show()
print("From the abvoe graph the rating of the most rooms from \033[1m {0} to {1} \033[0m ".format((df.describe()).iloc[(max_denisty("review_scores_rating")+3),5],(df.describe()).iloc[(max_denisty("review_scores_rating")+4),5]))
match max_denisty("review_scores_rating"):
        case 0:
            print("\033[1m So,Unfortunately the rate of the most rooms is low \033[0m") 
        case 3:
            print("\033[1m So,Great the rate of the most rooms is high \033[0m")
        case default:
            print("\033[1m So,Ok The rate of the most rooms is moderate \033[0m")


_,axis=plt.subplots(nrows=3,ncols=1,figsize=(15,30))
sns.countplot(data=df,y="neighbourhood_cleansed",ax=axis[0])
sns.countplot(data=df,y="property_type",ax=axis[1])
sns.countplot(data=df,x="room_type",ax=axis[2])
plt.show()
print("The rooms available in {0} neighbourhoods,the most rooms available in {1} ".format(len(df["neighbourhood_cleansed"].value_counts().index.tolist()),(df["neighbourhood_cleansed"].value_counts().index.tolist())[:9]))
print("The most property types of available rooms are {0}".format((df["property_type"].value_counts().index.tolist())[:5]))
print("The available room types are {0} and the most available type is {1} with {2} rooms".format((df["room_type"].value_counts().index.tolist()),(df["room_type"].value_counts().index.tolist())[0],(df["room_type"].value_counts())[0]))
print("The following graph shows the count of room types in each neighbourhood")

plt.figure(figsize=(15,30))
plt.title("Count of room_type vs neighbourhood")
sns.countplot(data=df,y="neighbourhood_cleansed",hue="room_type")
plt.show()

top_10_neighbourhood=pd.DataFrame(df["neighbourhood_cleansed"].value_counts()).T.columns
top_10_neighbourhood_data=df[df["neighbourhood_cleansed"].isin(top_10_neighbourhood[:10])]
plt.figure(figsize=(30,30))
plt.title("Price vs Top 10 neighbourhood")
sns.boxplot(x="neighbourhood_cleansed",y="price($)",data=df[df["neighbourhood_cleansed"].isin(top_10_neighbourhood[:10])])
plt.show()

s=(((df.groupby("neighbourhood_cleansed"))["price($)"]).agg(np.median)).sort_values(ascending=False)
print("The most expensive median price is {0}$ in {1} neighbourhood".format(s[0],s.index[0]))
print("The most cheap median price is {0}$ in {1} neighbourhood".format(s[-1],s.index[-1]))
s=(((top_10_neighbourhood_data.groupby("neighbourhood_cleansed"))["price($)"]).agg(np.median)).sort_values(ascending=False)
print("from above graph The most expensive median price in top 10 neighbourhoods that have rooms is {0}$ in {1} neighbourhood".format(s[0],s.index[0]))
print("from above graph The most cheap median price in top 10 neighbourhoods that have rooms is {0}$ in {1} neighbourhood".format(s[-1],s.index[-1]))

plt.figure(figsize=(30,30))
"""
sns.lmplot(y="latitude",x="longitude",data=df,hue="price range($)",fit_reg=False)
plt.show()
print("From above graph the most price range in the city is {0}".format(df["price range($)"].value_counts().index[0]))
"""

plt.figure(figsize=(30,30))
"""
sns.lmplot(y="latitude",x="longitude",data=df,hue="availability_365_range",fit_reg=False)
plt.show()

print("From above graph the most availability range in the city is {0} days".format(df["availability_365_range"].value_counts().index[0]))
"""

plt.figure(figsize=(30,30))

sns.lmplot(y="latitude",x="longitude",data=top_10_neighbourhood_data,hue="neighbourhood_cleansed",fit_reg=False)
plt.show()
print("Map of neighbourhoods")

plt.figure(figsize=(25,10))
plt.title("Count of room_type vs Top 10 neighbourhood")
sns.countplot(data=top_10_neighbourhood_data,x=top_10_neighbourhood_data["neighbourhood_cleansed"],hue=df["room_type"])
plt.show()

plt.figure(figsize=(30,15))
plt.title("Boxplot availability_365 vs top 10 neighbourhood")
sns.boxplot(data=top_10_neighbourhood_data, x=top_10_neighbourhood_data["neighbourhood_cleansed"],y=df['availability_365'])
plt.show()

plt.figure(figsize=(30,15))

sns.jointplot(x="price($)",y="availability_365",data=df)
plt.show()

pd.crosstab(df["bedrooms"],df["beds"])
pd.crosstab(df["neighbourhood_cleansed"],df["price range($)"])
bedrooms=pd.DataFrame(df.groupby("bedrooms")["price($)"].agg(np.mean))
bed=pd.DataFrame(df.groupby("beds")["price($)"].agg(np.mean))
plt.bar((bedrooms.T.columns).astype(str),bedrooms["price($)"])
plt.xlabel("bedrooms")
plt.ylabel("price($)")
plt.show()
_,axis=plt.subplots(nrows=1,ncols=2,sharey=True,figsize=(15,5))
sns.barplot(data=bedrooms,x=(bedrooms.T.columns).astype(str),y=bedrooms["price($)"],ax=axis[0])
sns.barplot(data=bed,x=(bed.T.columns).astype(str),y=bed["price($)"],ax=axis[1])
plt.show()

from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
x=df.loc[:,numerical_features]
df_scl=StandardScaler().fit_transform(x)
tsn=TSNE()
df_red=tsn.fit_transform(df_scl)
plt.figure(figsize=(25,10))
plt.title("Reduction data scattering")
sns.scatterplot(x=df_red[:,0],y=df_red[:,1])
plt.show()